# Module 2.4: Customizing the Elevator!

## Adding a Unique Element to the Elevator:
Now, it's time to add an extra mechanism to your elevator. You may add more motors, but you do not have to -- the only requirement is that you are doing *something* to make your creation more complex. Choose from the list of ideas below or come up with your own idea entirely!

Some possible ideas include:
* Adding a third floor that the elevator reaches
* Creating elevator doors that open and close
* Designing a mechanism to eject the minifigure from the elevator when it reaches the second floor

## Programming the Elevator:

Run the code to connect to SPIKE.

In [ ]:
import serial
import sys
import glob
import time
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


ser = None

def serial_ports():
     result = []
     ports = serial.tools.list_ports.comports()
     for port, desc, hwid in sorted(ports):
          comm =  "{}: {}".format(port, desc)
          result.append(comm) 
     return result

def InitSerial(port, bps = 9600, to = 0):
    global ser
    try:
        ser = serial.Serial(port, bps, timeout = to)  # open serial port
        ser.flushInput()
        ser.flushOutput()
        return ser.name
    except Exception as e:
        return 'ERR: ' + str(e)

def CloseSerial():
    return('done')
    try:
        ser.flush()
        ser.close()
        return 'done'
    except Exception as e:
        return 'ERR: ' + str(e)    

def WriteSerial(string):
    try:
        reply = ser.write(string.encode())
        return str(reply)
    except Exception as e:
        return 'ERR: ' + str(e)    

def ReadSerial():
    try:
        reply = ''
        while ser.in_waiting:
            reply = reply + ser.readline().decode()
        return reply
    except Exception as e:
        return 'ERR: ' +  str(e)
    
def SendRead(string):
    if WriteSerial(string):
        time.sleep(0.1)
        return ReadSerial()
    
updatedPortList = []

def search_for_ports():
#     updatedPortList = []
    serialPortList = serial_ports()
    for x in serialPortList:
        updatedPortList.append(x[:x.index(':')])
        
def on_value_change(change):
    with output2:
        InitSerial(change['new'],115200)
        WriteSerial('\x03')
        run_some_code()
        
def run_some_code():
    code = '''
import hub,utime
    '''
    WriteSerial('\x05')
    WriteSerial(code)
    WriteSerial('\x04')
    WriteSerial('\x03')
    
    time.sleep(1) #wait for everything to get over there and read all the replies
    words = ReadSerial()
#     print(words)
    if (">>>" in words):
        print("You are now connected to SPIKE Prime!")
    else:
        print("Error, try reconnecting or connecting to a different serial port.")
print ("done!")

updatedPortList = []
search_for_ports()
dropdown = widgets.Select(
            options=updatedPortList,
            description='Select Port:',
            disabled=False)
output2 = widgets.Output()
display(dropdown, output2)
dropdown.observe(on_value_change, names='value')

Load the move and buttonPressed functions.

In [ ]:
# the move function turns a motor for a specified number of rotations
def move(port,rotations,speed): # takes the port letter (ex. 'A'), number of motor rotations, and speed (between -100 and 100) as parameters
    moveCode = """hub.port."""+str(port)+""".motor.run_for_degrees("""+str(360*rotations)+""", """+str(speed)+""")"""
    WriteSerial('\x05')
    WriteSerial(moveCode)
    WriteSerial('\x04')

# the buttonPressed function returns true/false depending on whether or not the button has been pressed recently
def buttonPressed(buttonType): # only parameter is the button ("left" or "right")
    if(buttonType=="right"):
        buttonCode = """print(hub.button.right.was_pressed())"""
    elif(buttonType=="left"):
        buttonCode = """print(hub.button.left.was_pressed())"""
    WriteSerial('\x05')
    WriteSerial(buttonCode)
    WriteSerial('\x04')
    responses = ReadSerial()
    if("True" in responses):
        return True
    else:
        return False

Use the cell below to program your elevator.

### Congratulations, you've completed the second module! 
When you are finished with your elevator, disassemble it. You're now ready to move on to the next module! 